SMB、HML两个指标的计算方式如下：
1. 首先，按市值大小平均分为两组(Small 组, Big 组)，基准是这一时间的市场上公司市值中位数;
2. 按 BM 从小到大分三组,即前 30%(Growth 组),中间 40%(Neutral 组),后 30%(Value 组)；
3. 每个组的月回报以组内所有成员股票当月回报的加权平均数为依据,计算每个月的 SMB 和 HML 值。具体计算公式如下: 
   SMB = Small size return - Big size return)
   HML = Value company return - Growth company return
4. 以市场回报减去无风险回报,即得到超额市场回报(Rm-Rf)。（由于Ricequant平台无法提取国债收益率，我只能对这个时段的所有IBO1M求平均值作为无风    险回报率

In [1]:
%pdb

Automatic pdb calling has been turned ON


In [2]:
# %load fama.py
import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
# df_book_value=pd.read_csv("./book_value_per_share.csv",index_col=0)
# df_market_cap=pd.read_csv("./market_cap.csv",index_col=0)
# df_price_data=pd.read_csv("./price_data.csv",index_col=0)

# index_date=df_book_value.index
# stocks=df_book_value.columns
# format='%Y-%m-%d %H:%M:%S'
# pd.to_datetime(research_endtime)

In [3]:
research_starttime='2010-04-01'
research_endtime='2018-04-01'
interval_month=24
# date_range=pd.date_range(research_starttime,research_endtime)
# len(date_range)/31
# pd.to_datetime(research_endtime)-(pd.to_datetime(research_endtime)-pd.to_datetime(research_starttime))/4
# print(datetime.date.today() +relativedelta(months=-1))
step0=pd.to_datetime(research_endtime)
step1=pd.to_datetime(research_endtime)-relativedelta(months=interval_month*1)
step2=pd.to_datetime(research_endtime)-relativedelta(months=interval_month*2)
step3=pd.to_datetime(research_endtime)-relativedelta(months=interval_month*3)
step4=pd.to_datetime(research_endtime)-relativedelta(months=interval_month*4)

In [4]:
def get_years_fundamentals(interval_month,enddate):
    return get_fundamentals(
        query(fundamentals.eod_derivative_indicator.market_cap,fundamentals.financial_indicator.book_value_per_share)
        ,enddate, str(interval_month)+'m')
#   ,fundamentals.financial_indicator.book_value_per_share

In [5]:
dp_arr = [get_years_fundamentals(interval_month,step0),
          get_years_fundamentals(interval_month,step1),
          get_years_fundamentals(interval_month,step2),
          get_years_fundamentals(interval_month,step3)]
dp_arr



[<class 'pandas.core.panel.Panel'>
 Dimensions: 2 (items) x 24 (major_axis) x 3511 (minor_axis)
 Items axis: market_cap to book_value_per_share
 Major_axis axis: 2018-03-30 00:00:00 to 2016-04-06 00:00:00
 Minor_axis axis: 300407.XSHE to 300444.XSHE, <class 'pandas.core.panel.Panel'>
 Dimensions: 2 (items) x 24 (major_axis) x 2841 (minor_axis)
 Items axis: market_cap to book_value_per_share
 Major_axis axis: 2016-04-01 00:00:00 to 2014-04-09 00:00:00
 Minor_axis axis: 300407.XSHE to 300444.XSHE, <class 'pandas.core.panel.Panel'>
 Dimensions: 2 (items) x 24 (major_axis) x 2521 (minor_axis)
 Items axis: market_cap to book_value_per_share
 Major_axis axis: 2014-04-01 00:00:00 to 2012-04-11 00:00:00
 Minor_axis axis: 300034.XSHE to 000517.XSHE, <class 'pandas.core.panel.Panel'>
 Dimensions: 2 (items) x 24 (major_axis) x 2372 (minor_axis)
 Items axis: market_cap to book_value_per_share
 Major_axis axis: 2012-03-30 00:00:00 to 2010-04-02 00:00:00
 Minor_axis axis: 300034.XSHE to 000517.XSHE]

In [6]:
# ,ignore_index=True
dp_merge=pd.concat(dp_arr,axis=1)
dp_merge
  

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 96 (major_axis) x 3529 (minor_axis)
Items axis: market_cap to book_value_per_share
Major_axis axis: 2018-03-30 00:00:00 to 2010-04-02 00:00:00
Minor_axis axis: 000001.XSHE to 603999.XSHG

In [7]:
df_book_value=dp_merge["book_value_per_share"]
df_book_value

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2018-03-30,11.77,12.02,1.374,1.1962,4.1242,1.0991,2.2153,2.1635,2.7179,4.9023,...,3.7975,8.2268,5.8856,5.0689,5.9552,1.7665,4.732,2.6177,2.3195,2.8958
2018-02-28,11.54,10.5418,1.374,1.1962,3.7787,1.0991,2.2153,2.1635,2.7179,4.8304,...,3.7975,8.2268,5.8856,5.0689,5.9552,1.7259,4.732,2.6177,2.3195,2.8958
2018-01-26,11.54,10.5418,1.374,1.1962,3.7787,1.0991,2.2153,2.1635,2.7179,4.8304,...,3.7975,8.2268,5.8856,5.0689,5.9552,1.7259,4.732,2.6177,2.3195,2.8958
2017-12-26,11.54,10.5418,1.374,1.1962,3.7787,1.0991,2.2153,2.1635,2.7179,4.8304,...,3.7975,8.2268,5.8856,5.0689,5.9552,1.7259,4.732,2.6177,2.3195,2.8958
2017-11-24,11.54,10.5418,1.374,1.1962,3.7787,1.0991,2.2153,2.1635,2.7179,4.8304,...,3.7975,8.2268,5.8856,5.0689,5.9552,1.7259,4.732,2.6177,2.3195,2.8958
2017-10-24,11.54,10.2,1.3938,1.1948,3.7124,1.13,2.1955,2.1101,2.6469,4.4732,...,3.6819,8.1476,5.6548,4.8764,5.9552,1.1078,4.5464,2.4926,2.2511,5.7284
2017-09-22,11.15,10.2,1.3938,1.1948,3.7124,1.13,2.1955,2.1101,2.6469,4.4732,...,3.6819,8.1476,5.6548,4.8764,5.8603,1.1078,4.5464,2.4926,2.2511,5.7284
2017-08-22,11.15,10.3242,1.4247,1.202,3.7124,1.6052,2.1838,2.1071,2.6713,4.5512,...,5.5301,8.1874,5.6548,4.8764,5.8603,1.1448,4.5464,3.8168,2.1853,5.7564
2017-07-21,10.94,10.3242,1.4247,1.202,3.7844,1.6052,2.1838,2.1071,2.6713,4.5512,...,5.5301,8.1874,6.1934,4.8975,5.7729,1.1448,4.6558,3.8168,2.1853,5.7564
2017-06-21,10.94,10.3242,1.4247,1.202,3.7844,1.6052,2.1838,2.1071,2.6713,4.5512,...,5.5301,8.1874,6.1934,4.8975,5.7729,1.1448,4.6558,3.8168,2.1853,5.7564


In [8]:
df_market_cap=dp_merge["market_cap"]
df_market_cap

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2018-03-30,1.87157e+11,3.67493e+11,1.91887e+09,4.08595e+09,9.94946e+09,4.9646e+09,1.92774e+10,1.30465e+10,3.69754e+09,9.53567e+09,...,3.78837e+09,3.8472e+09,1.1805e+10,3.1835e+09,1.88126e+09,1.83162e+11,5.07854e+09,8.09266e+09,4.06373e+09,4.2336e+09
2018-02-28,2.06903e+11,3.6098e+11,1.72992e+09,4.04361e+09,1.32975e+10,5.77182e+09,2.13066e+10,1.2982e+10,3.64835e+09,9.24364e+09,...,3.54549e+09,3.312e+09,1.1214e+10,2.78697e+09,1.69791e+09,1.85321e+11,5.03652e+09,6.86186e+09,4.06373e+09,4.12416e+09
2018-01-26,2.41244e+11,4.42008e+11,1.81726e+09,4.47761e+09,1.32975e+10,5.77182e+09,2.34203e+10,1.45726e+10,4.42722e+09,1.06799e+10,...,3.842e+09,4.9032e+09,1.1124e+10,2.91321e+09,2.0594e+09,1.68042e+11,5.31566e+09,7.2072e+09,4.83511e+09,4.77504e+09
2017-12-26,2.34548e+11,3.36694e+11,1.88444e+09,4.34e+09,1.32975e+10,5.77182e+09,2.46604e+10,1.54538e+10,4.5174e+09,9.98861e+09,...,3.76945e+09,5.8344e+09,1.1511e+10,2.82177e+09,2.05791e+09,1.38667e+11,5.64282e+09,7.2198e+09,4.83511e+09,4.28544e+09
2017-11-24,2.50001e+11,3.51045e+11,2.30516e+09,4.45644e+09,1.32975e+10,5.77182e+09,2.41531e+10,1.6378e+10,4.43541e+09,9.89921e+09,...,3.98079e+09,6.168e+09,1.2081e+10,2.82258e+09,2.37319e+09,1.44715e+11,5.39069e+09,7.3332e+09,5.11522e+09,4.5504e+09
2017-10-24,1.95571e+11,2.98057e+11,2.09942e+09,5.16566e+09,1.32975e+10,5.77182e+09,2.50268e+10,1.8205e+10,5.67339e+09,1.14607e+10,...,5.20783e+09,5.5176e+09,1.1163e+10,3.07046e+09,2.55655e+09,1.55947e+11,5.04852e+09,8.1648e+09,6.13654e+09,5.69088e+09
2017-09-22,1.9643e+11,3.06999e+11,2.03811e+09,5.41971e+09,1.32975e+10,5.37341e+09,2.55549e+10,1.9602e+10,4.13207e+09,1.1854e+10,...,4.76307e+09,5.6568e+09,1.1148e+10,3.2307e+09,2.65345e+09,1.64154e+11,4.97048e+09,8.064e+09,6.54162e+09,6.0768e+09
2017-08-22,1.82865e+11,2.56219e+11,1.92559e+09,5.11273e+09,1.12455e+10,5.54317e+09,2.27747e+10,1.82479e+10,4.00909e+09,1.1413e+10,...,4.7126e+09,5.7528e+09,1.0755e+10,2.83495e+09,2.57369e+09,1.44067e+11,5.0095e+09,7.6419e+09,5.58494e+09,5.76576e+09
2017-07-21,1.86986e+11,2.7355e+11,1.8055e+09,5.19742e+09,1.14345e+10,5.56049e+09,2.15953e+10,1.799e+10,3.9353e+09,1.16395e+10,...,4.36562e+09,5.0088e+09,1.1385e+10,2.83495e+09,2.58487e+09,1.2429e+11,4.61631e+09,6.6969e+09,5.45135e+09,5.62752e+09
2017-06-21,1.57109e+11,2.32264e+11,2.23126e+09,5.69493e+09,1.1826e+10,5.49929e+09,2.07029e+10,1.68939e+10,4.28784e+09,1.25573e+10,...,5.08481e+09,5.1696e+09,1.1724e+10,3.29868e+09,3.19681e+09,8.40982e+10,5.06653e+09,9.1854e+09,6.17468e+09,6.24672e+09


计算说明：
dp_merge.major_xs("2018-03-30")


In [9]:
index_date = df_book_value.index
print("index_date",index_date)
stocks = df_book_value.columns
stocks

index_date DatetimeIndex(['2018-03-30', '2018-02-28', '2018-01-26', '2017-12-26',
               '2017-11-24', '2017-10-24', '2017-09-22', '2017-08-22',
               '2017-07-21', '2017-06-21', '2017-05-19', '2017-04-19',
               '2017-03-17', '2017-02-17', '2017-01-17', '2016-12-16',
               '2016-11-16', '2016-10-14', '2016-09-14', '2016-08-12',
               '2016-07-12', '2016-06-08', '2016-05-06', '2016-04-06',
               '2016-04-01', '2016-03-01', '2016-02-01', '2015-12-31',
               '2015-11-30', '2015-10-30', '2015-09-30', '2015-08-28',
               '2015-07-28', '2015-06-26', '2015-05-26', '2015-04-24',
               '2015-03-24', '2015-02-17', '2015-01-16', '2014-12-16',
               '2014-11-14', '2014-10-14', '2014-09-12', '2014-08-12',
               '2014-07-11', '2014-06-11', '2014-05-09', '2014-04-09',
               '2014-04-01', '2014-02-28', '2014-01-28', '2013-12-27',
               '2013-11-27', '2013-10-25', '2013-09-25', '2013-08-

Index(['000001.XSHE', '000002.XSHE', '000004.XSHE', '000005.XSHE',
       '000006.XSHE', '000007.XSHE', '000008.XSHE', '000009.XSHE',
       '000010.XSHE', '000011.XSHE',
       ...
       '603987.XSHG', '603988.XSHG', '603989.XSHG', '603990.XSHG',
       '603991.XSHG', '603993.XSHG', '603996.XSHG', '603997.XSHG',
       '603998.XSHG', '603999.XSHG'],
      dtype='object', length=3529)

In [10]:
data_endtime=index_date[0]
data_count=len(index_date)
data_starttime=index_date[data_count-1]


In [11]:
# index_date

In [12]:
df_market = all_instruments(type='CS')
print("df_market['order_book_id']=",len(df_market['order_book_id']),df_market['order_book_id']);

df_market['order_book_id']= 3631 0       603186.XSHG
1       000062.XSHE
2       300643.XSHE
3       600315.XSHG
4       300146.XSHE
5       000990.XSHE
6       300631.XSHE
7       300363.XSHE
8       600551.XSHG
9       603656.XSHG
10      002245.XSHE
11      601113.XSHG
12      002440.XSHE
13      300285.XSHE
14      300052.XSHE
15      002543.XSHE
16      002128.XSHE
17      600497.XSHG
18      600631.XSHG
19      603901.XSHG
20      000539.XSHE
21      600597.XSHG
22      600646.XSHG
23      603099.XSHG
24      000831.XSHE
25      600481.XSHG
26      000697.XSHE
27      000921.XSHE
28      603877.XSHG
29      601268.XSHG
           ...     
3601    600292.XSHG
3602    002712.XSHE
3603    300719.XSHE
3604    603916.XSHG
3605    600878.XSHG
3606    600197.XSHG
3607    300176.XSHE
3608    300056.XSHE
3609    002184.XSHE
3610    300641.XSHE
3611    002915.XSHE
3612    000725.XSHE
3613    002439.XSHE
3614    300006.XSHE
3615    600161.XSHG
3616    000584.XSHE
3617    603878.XSHG
3618   

In [13]:
df_price_data = get_price(list(df_market['order_book_id']),start_date=data_starttime, end_date=data_endtime,frequency='1d',fields='close')

In [14]:
#save data to reuse offline
# df_book_value.to_csv("book_value_per_share.csv")
# df_market_cap.to_csv("market_cap.csv")
# df_market.to_csv("all_instruments.csv")
# df_price_data.to_csv("price_data.csv")

In [15]:
df_price_data

,603186.XSHG,000062.XSHE,300643.XSHE,600315.XSHG,300146.XSHE,000990.XSHE,300631.XSHE,300363.XSHE,600551.XSHG,603656.XSHG,...,601857.XSHG,000825.XSHE,000543.XSHE,002917.XSHE,000569.XSHE,000732.XSHE,600696.XSHG,603288.XSHG,002725.XSHE,002246.XSHE
2010-04-02,NaN,9.6163,NaN,18.0081,NaN,14.8820,NaN,NaN,15.5627,NaN,...,10.5334,7.7195,4.9641,NaN,NaN,1.9866,7.22,NaN,NaN,11.6093
2010-04-06,NaN,9.5244,NaN,18.5700,NaN,15.1620,NaN,NaN,15.2372,NaN,...,10.5334,7.6462,5.0109,NaN,NaN,1.9866,7.17,NaN,NaN,11.7370
2010-04-07,NaN,9.4234,NaN,18.5747,NaN,15.5483,NaN,NaN,15.3205,NaN,...,10.4766,7.8111,4.9589,NaN,NaN,1.9866,7.22,NaN,NaN,11.9540
2010-04-08,NaN,9.4693,NaN,19.1133,NaN,15.6449,NaN,NaN,15.8049,NaN,...,10.3548,7.5913,4.8860,NaN,NaN,1.9866,7.13,NaN,NaN,11.7498
2010-04-09,NaN,9.6347,NaN,19.2201,NaN,15.7801,NaN,NaN,15.8503,NaN,...,10.3954,7.6462,4.9173,NaN,NaN,1.9866,7.15,NaN,NaN,12.1774
2010-04-12,NaN,9.7724,NaN,19.9120,NaN,16.1567,NaN,NaN,16.4786,NaN,...,10.3548,7.4173,4.8444,NaN,NaN,1.9866,7.00,NaN,NaN,12.1774
2010-04-13,NaN,9.4785,NaN,20.1860,NaN,15.6932,NaN,NaN,16.4862,NaN,...,10.5902,7.4997,4.7612,NaN,NaN,1.9866,6.87,NaN,NaN,11.9795
2010-04-14,NaN,10.4245,NaN,19.8335,NaN,15.7608,NaN,NaN,17.1674,NaN,...,10.5253,7.5089,4.7872,NaN,NaN,1.9866,6.94,NaN,NaN,11.8455
2010-04-15,NaN,10.1857,NaN,19.4014,NaN,15.9153,NaN,NaN,16.7662,NaN,...,10.7363,7.4081,4.7351,NaN,NaN,1.9866,6.77,NaN,NaN,11.6349
2010-04-16,NaN,9.9561,NaN,20.1074,NaN,16.6106,NaN,NaN,16.4634,NaN,...,10.5821,7.3440,4.7039,NaN,NaN,1.9866,6.71,NaN,NaN,11.7306


In [16]:
df_book_value1=df_book_value
df_market_cap1=df_market_cap
df_price_data1=df_price_data
df_book_value=df_book_value.fillna(0)
df_market_cap=df_market_cap.fillna(0)
df_price_data=df_price_data.fillna(0)

In [17]:
btm = pd.DataFrame(index=index_date,columns=stocks)
i=0
for stk in stocks:
    for date in index_date:
        i=i+1
        if i==1:
            print("type(stk)=",type(stk),stk,"type(date)=",type(date),date)
        else:
            pass
#         dt=date.strftime(format)    
#         print("dt",type(dt),dt)
        is_valid=False
        if(type(date)==str):
            is_valid=hasattr(df_price_data[stk],date)
        else:
            is_valid=hasattr(df_price_data[stk],str(data_endtime))
          
        if is_valid:
            if df_price_data[stk][date]!=0:
                btm[stk][date] = df_book_value[stk][date]/df_price_data[stk][date]
            else:
                btm[stk][date] = 0
        else:
#             print('hasattr error',stk,date,type(date))
            btm[stk][date]=0
            
btm

type(stk)= <class 'str'> 000001.XSHE type(date)= <class 'pandas.tslib.Timestamp'> 2018-03-30 00:00:00


,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2018-03-30,1.07982,0.361069,0.0601313,0.309896,0.559593,0.0766992,0.323874,0.356425,0.602639,0.306394,...,0.316195,0.362312,0.149571,0.129482,0.235943,0.208314,0.279669,0.211296,0.24597,0.393986
2018-02-28,0.957676,0.322379,0.066699,0.313141,0.383624,0.0659724,0.293029,0.358195,0.610764,0.311438,...,0.337856,0.42086,0.157453,0.147904,0.261422,0.201154,0.282002,0.249196,0.24597,0.404441
2018-01-26,0.821352,0.263282,0.0634935,0.28279,0.383624,0.0659724,0.266582,0.3191,0.503315,0.269554,...,0.311782,0.284282,0.158727,0.141495,0.215534,0.221838,0.267194,0.234382,0.206729,0.349312
2017-12-26,0.844802,0.345633,0.0612299,0.291756,0.383624,0.0659724,0.253177,0.300904,0.493267,0.28821,...,0.317782,0.238909,0.153391,0.14608,0.21569,0.268832,0.251702,0.233974,0.206729,0.38922
2017-11-24,0.792582,0.331503,0.0500546,0.284133,0.383624,0.0659724,0.258495,0.283924,0.502384,0.290813,...,0.300911,0.225987,0.146153,0.146039,0.187035,0.257597,0.263474,0.230354,0.195409,0.366557
2017-10-24,1.01317,0.377778,0.055752,0.244836,0.376893,0.0678271,0.247241,0.249126,0.3825,0.232616,...,0.22301,0.250193,0.15197,0.129161,0.173621,0.153435,0.270297,0.197006,0.158083,0.579798
2017-09-22,0.97465,0.366775,0.0574289,0.233359,0.376893,0.0728562,0.241264,0.231371,0.525179,0.224897,...,0.243834,0.244037,0.152174,0.122755,0.164615,0.145763,0.274541,0.199469,0.148294,0.542976
2017-08-22,1.04695,0.459949,0.0621326,0.248861,0.445666,0.100325,0.269273,0.248186,0.546278,0.237661,...,0.370154,0.241136,0.157735,0.139891,0.169716,0.171634,0.272403,0.322308,0.168619,0.575065
2017-07-21,1.00459,0.430809,0.0662651,0.244807,0.4468,0.100012,0.283979,0.252323,0.556521,0.233036,...,0.399574,0.276954,0.163199,0.140496,0.166462,0.155543,0.302718,0.367789,0.172751,0.589191
2017-06-21,1.2131,0.507387,0.0536206,0.22342,0.432009,0.101126,0.296309,0.268698,0.510765,0.216004,...,0.343058,0.26834,0.15848,0.120745,0.134598,0.22988,0.275818,0.268147,0.153369,0.533178


In [18]:
# btm.to_csv("btm.csv")


In [19]:
##calculate market size medians
median_size = {}
for date in index_date:
    #日市值中位数
    median_size[date] = np.median(df_market_cap.loc[date])

In [20]:
median_size

{Timestamp('2010-04-02 00:00:00'): 1115692380.0,
 Timestamp('2010-05-04 00:00:00'): 1217365800.0,
 Timestamp('2010-06-04 00:00:00'): 1284827046.0,
 Timestamp('2010-07-06 00:00:00'): 1276817600.0,
 Timestamp('2010-08-06 00:00:00'): 1581132464.0,
 Timestamp('2010-09-08 00:00:00'): 1873956325.0,
 Timestamp('2010-10-08 00:00:00'): 1903470000.0,
 Timestamp('2010-11-10 00:00:00'): 2191678200.0,
 Timestamp('2010-12-10 00:00:00'): 2178757380.0,
 Timestamp('2011-01-11 00:00:00'): 2190180000.0,
 Timestamp('2011-02-11 00:00:00'): 2236410000.0,
 Timestamp('2011-03-11 00:00:00'): 2456554264.0,
 Timestamp('2011-04-13 00:00:00'): 2553594732.0,
 Timestamp('2011-05-13 00:00:00'): 2438693185.0,
 Timestamp('2011-06-15 00:00:00'): 2263200000.0,
 Timestamp('2011-07-15 00:00:00'): 2533650000.0,
 Timestamp('2011-08-16 00:00:00'): 2427732000.0,
 Timestamp('2011-09-16 00:00:00'): 2372160000.0,
 Timestamp('2011-10-18 00:00:00'): 2215618014.0,
 Timestamp('2011-11-18 00:00:00'): 2324675466.0,
 Timestamp('2011-12-

In [21]:
##calculate the 30%, 70% quantile of book to market ratio each month

lmark = {}
hmark = {}
for date in index_date:
    lmark[date] = np.percentile(list(btm.loc[date]),30)
    hmark[date] = np.percentile(list(btm.loc[date]),70)
    

In [22]:
lmark

{Timestamp('2010-04-02 00:00:00'): 0.0,
 Timestamp('2010-05-04 00:00:00'): 0.0,
 Timestamp('2010-06-04 00:00:00'): 0.0,
 Timestamp('2010-07-06 00:00:00'): 0.0,
 Timestamp('2010-08-06 00:00:00'): 0.0,
 Timestamp('2010-09-08 00:00:00'): 0.0,
 Timestamp('2010-10-08 00:00:00'): 0.0,
 Timestamp('2010-11-10 00:00:00'): 0.0,
 Timestamp('2010-12-10 00:00:00'): 0.0,
 Timestamp('2011-01-11 00:00:00'): 0.0,
 Timestamp('2011-02-11 00:00:00'): 0.0,
 Timestamp('2011-03-11 00:00:00'): 0.0,
 Timestamp('2011-04-13 00:00:00'): 0.0,
 Timestamp('2011-05-13 00:00:00'): 0.0,
 Timestamp('2011-06-15 00:00:00'): 0.0,
 Timestamp('2011-07-15 00:00:00'): 0.0,
 Timestamp('2011-08-16 00:00:00'): 0.0,
 Timestamp('2011-09-16 00:00:00'): 0.0,
 Timestamp('2011-10-18 00:00:00'): 0.0,
 Timestamp('2011-11-18 00:00:00'): 0.0,
 Timestamp('2011-12-20 00:00:00'): 0.0,
 Timestamp('2012-01-20 00:00:00'): 0.0,
 Timestamp('2012-02-29 00:00:00'): 0.0,
 Timestamp('2012-03-30 00:00:00'): 0.0,
 Timestamp('2012-04-11 00:00:00'): 0.0,


In [23]:
hmark

{Timestamp('2010-04-02 00:00:00'): 0.2940647877556255,
 Timestamp('2010-05-04 00:00:00'): 0.34648643565853487,
 Timestamp('2010-06-04 00:00:00'): 0.39395407553587325,
 Timestamp('2010-07-06 00:00:00'): 0.44345648964051532,
 Timestamp('2010-08-06 00:00:00'): 0.38352805677264884,
 Timestamp('2010-09-08 00:00:00'): 0.35219487115898962,
 Timestamp('2010-10-08 00:00:00'): 0.35947268066508281,
 Timestamp('2010-11-10 00:00:00'): 0.33469493969094366,
 Timestamp('2010-12-10 00:00:00'): 0.38130519931656598,
 Timestamp('2011-01-11 00:00:00'): 0.39883960181728334,
 Timestamp('2011-02-11 00:00:00'): 0.40968892354386832,
 Timestamp('2011-03-11 00:00:00'): 0.39342308361046774,
 Timestamp('2011-04-13 00:00:00'): 0.41837406108134989,
 Timestamp('2011-05-13 00:00:00'): 0.47437978730355645,
 Timestamp('2011-06-15 00:00:00'): 0.53047339960015905,
 Timestamp('2011-07-15 00:00:00'): 0.49680219223924876,
 Timestamp('2011-08-16 00:00:00'): 0.53324369140537797,
 Timestamp('2011-09-16 00:00:00'): 0.547778725531

In [24]:
#暂时先保留对数据对修正处理
# for date in index_date:
#     if lmark[date] == 0:
#         lmark[date]=hmark[date]/2

In [25]:
lmark

{Timestamp('2010-04-02 00:00:00'): 0.0,
 Timestamp('2010-05-04 00:00:00'): 0.0,
 Timestamp('2010-06-04 00:00:00'): 0.0,
 Timestamp('2010-07-06 00:00:00'): 0.0,
 Timestamp('2010-08-06 00:00:00'): 0.0,
 Timestamp('2010-09-08 00:00:00'): 0.0,
 Timestamp('2010-10-08 00:00:00'): 0.0,
 Timestamp('2010-11-10 00:00:00'): 0.0,
 Timestamp('2010-12-10 00:00:00'): 0.0,
 Timestamp('2011-01-11 00:00:00'): 0.0,
 Timestamp('2011-02-11 00:00:00'): 0.0,
 Timestamp('2011-03-11 00:00:00'): 0.0,
 Timestamp('2011-04-13 00:00:00'): 0.0,
 Timestamp('2011-05-13 00:00:00'): 0.0,
 Timestamp('2011-06-15 00:00:00'): 0.0,
 Timestamp('2011-07-15 00:00:00'): 0.0,
 Timestamp('2011-08-16 00:00:00'): 0.0,
 Timestamp('2011-09-16 00:00:00'): 0.0,
 Timestamp('2011-10-18 00:00:00'): 0.0,
 Timestamp('2011-11-18 00:00:00'): 0.0,
 Timestamp('2011-12-20 00:00:00'): 0.0,
 Timestamp('2012-01-20 00:00:00'): 0.0,
 Timestamp('2012-02-29 00:00:00'): 0.0,
 Timestamp('2012-03-30 00:00:00'): 0.0,
 Timestamp('2012-04-11 00:00:00'): 0.0,


In [26]:
#是否需要做价格回溯，即不存在时，往前找
#return of each stock
return_data = pd.DataFrame(index=index_date,columns=stocks)
lastIndex=len(index_date)-1;
for stk in stocks:
    i = 0
    for date in index_date:
        if i+1<=lastIndex and hasattr(df_price_data[stk],str(index_date[i+1])):
            if df_price_data[stk][index_date[i+1]]==0: #倒序的价格，取出昨日价格，即下一行的数据，第二行为0，不能计算
                return_data[stk][date] = 0
            else: 
                if hasattr(df_price_data[stk],str(index_date[i])) and df_price_data[stk][index_date[i]]!=0:#第一行有数据
                    return_data[stk][date] = df_price_data[stk][index_date[i]]/df_price_data[stk][index_date[i+1]] - 1
                else:
                    #无当天数据，不加这一句的话，打出来为NaN，#2010-11-01 所有数据都是NaN,因为price_date的数据只取到2010-10-29
                    return_data[stk][date]=0
#                     print('hasattr error,index_date[i]',stk,date,type(date))
        else:
            return_data[stk][date]=0
            #无上一交易日数据，即第二行，最后一个数据
#             print('hasattr error,index_date[i+1]',stk,date,type(date))
        i = i + 1
        

In [27]:
return_data

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2018-03-30,-0.0954357,0.0180428,0.109223,0.0104712,-0.251777,-0.139856,-0.0952381,0.00496689,0.0134831,0.0315925,...,0.0685053,0.161595,0.052702,0.142276,0.107989,-0.011655,0.00834327,0.179369,0,0.0265363
2018-02-28,-0.142349,-0.183317,-0.0480591,-0.0969267,0,0,-0.0902527,-0.109145,-0.175926,-0.134487,...,-0.0771757,-0.324522,0.00809061,-0.0433342,-0.175534,0.102828,-0.0525127,-0.059444,-0.159537,-0.136309
2018-01-26,0.0285505,0.312787,-0.0356506,0.0317073,0,0,-0.0502857,-0.0570236,-0.0199637,0.0692124,...,0.0192469,-0.159606,-0.03362,0.0324069,0.000724375,0.211838,-0.0579787,-0.00174294,0,0.114247
2017-12-26,-0.0618132,-0.0408805,-0.182514,-0.0261283,0,0,0.0210035,-0.0564304,0.0184843,0.0090307,...,-0.0530903,-0.054085,-0.0471815,-0.000285226,-0.132852,-0.041791,0.0467706,-0.0154702,-0.0547599,-0.0582278
2017-11-24,0.278314,0.177778,0.098,-0.137295,0,0,-0.0349099,-0.100354,-0.218208,-0.136245,...,-0.235615,0.117874,0.082236,-0.0806555,-0.0717201,-0.0720222,0.0677765,-0.101846,-0.166433,-0.200405
2017-10-24,-0.00437063,-0.0291262,0.0300783,-0.046875,0,0.0741457,-0.0241758,-0.0712719,0.373016,-0.0331825,...,0.0933775,-0.0246085,0.00134553,-0.0495989,-0.0365169,-0.05,0.0157005,0.0124998,-0.0619236,-0.0635071
2017-09-22,0.0741784,0.238951,0.0584387,0.0600414,0.182473,-0.030625,0.122072,0.0742049,0.0306748,0.0386423,...,0.0107095,-0.0166876,0.0365411,0.139595,0.0309875,0.13943,-0.0077891,0.0552351,0.171296,0.0539461
2017-08-22,-0.0220386,-0.0633557,0.0665116,-0.0162933,-0.0165289,-0.00311526,0.0546164,0.0166691,0.01875,-0.0194572,...,0.0794798,0.148537,-0.055336,0,-0.00432526,-0.09375,0.0851756,0.14111,0.0245059,0.024565
2017-07-21,0.207558,0.177754,-0.190817,-0.0873606,-0.033105,0.0111319,0.0434193,0.064895,-0.082218,-0.0730897,...,-0.141439,-0.0311032,-0.028915,-0.14058,-0.19142,0.477912,-0.0888626,-0.270922,-0.112193,-0.0950687
2017-06-21,0.0529376,0.0975495,-0.261944,0.0247619,0.019387,-6.29985e-06,0.00278931,0.0102807,-0.0368324,0.147691,...,-0.085063,-0.001881,0.0341415,0.0202693,0.00775376,0.152778,0.0521001,-0.162356,0.00920056,-0.0100404


In [28]:
# return_data.to_csv("return_data.csv")

In [29]:
##calculate SMB and HML
smb = pd.Series(index = index_date)
hml = pd.Series(index = index_date)
first_row = True
for date in index_date:
#     if first_row:
#         smb[date]=0
#         hml[date]=0
#         first_row=False
#         continue
    small_size=0.0
    big_size=0.0
    value_btm=0.0
    middle_btm=0.0
    growth_btm=0.0
    for stk in stocks:
        #计算市值size分组
        if df_market_cap[stk][date]<median_size[date]:#判定为小市值
            small_size = small_size + return_data[stk][date]*df_market_cap[stk][date]#计算每日总的回报＝回报率＊市值
        else: 
            big_size = big_size + return_data[stk][date]*df_market_cap[stk][date]
        #计算btm分组
        if btm[stk][date]<=lmark[date]:
            growth_btm = growth_btm + return_data[stk][date]*df_market_cap[stk][date]
        elif btm[stk][date]>hmark[date]:
            value_btm = value_btm + return_data[stk][date]*df_market_cap[stk][date]
        else:
            middle_btm = middle_btm + return_data[stk][date]*df_market_cap[stk][date]

    mktcap = np.sum(df_market_cap.loc[date])
    smb[date] = (small_size - big_size)/mktcap#每日回报率＝每日总回报/总市值
    hml[date] = (value_btm - growth_btm)/mktcap

In [30]:
smb

2018-03-30    0.015333
2018-02-28    0.052135
2018-01-26   -0.078953
2017-12-26   -0.000073
2017-11-24   -0.017186
2017-10-24   -0.022748
2017-09-22   -0.022873
2017-08-22   -0.023196
2017-07-21   -0.043478
2017-06-21   -0.033707
2017-05-19    0.005097
2017-04-19   -0.006285
2017-03-17   -0.021742
2017-02-17   -0.023655
2017-01-17    0.001696
2016-12-16    0.012231
2016-11-16   -0.032798
2016-10-14   -0.018877
2016-09-14    0.008237
2016-08-12   -0.011577
2016-07-12   -0.056178
2016-06-08   -0.021526
2016-05-06    0.031994
2016-04-06   -0.013960
2016-04-01   -0.090057
2016-03-01   -0.016934
2016-02-01    0.178821
2015-12-31   -0.032417
2015-11-30   -0.042175
2015-10-30   -0.112353
                ...   
2012-09-14    0.003623
2012-08-14    0.012315
2012-07-13    0.031216
2012-06-13    0.011132
2012-05-11   -0.037257
2012-04-11   -0.021134
2012-03-30    0.055471
2012-02-29   -0.055798
2012-01-20   -0.039975
2011-12-20    0.067964
2011-11-18   -0.017607
2011-10-18    0.034397
2011-09-16 

In [31]:
hml

2018-03-30   -0.035067
2018-02-28   -0.033533
2018-01-26    0.045621
2017-12-26   -0.014178
2017-11-24   -0.009724
2017-10-24   -0.009493
2017-09-22   -0.002454
2017-08-22   -0.005136
2017-07-21    0.029293
2017-06-21    0.013257
2017-05-19   -0.012410
2017-04-19   -0.004103
2017-03-17   -0.019800
2017-02-17    0.011954
2017-01-17    0.005375
2016-12-16   -0.008769
2016-11-16    0.014014
2016-10-14    0.007655
2016-09-14   -0.011226
2016-08-12    0.009449
2016-07-12    0.013738
2016-06-08    0.002139
2016-05-06   -0.020136
2016-04-06    0.004245
2016-04-01    0.048811
2016-03-01    0.006572
2016-02-01   -0.108849
2015-12-31    0.013264
2015-11-30    0.004794
2015-10-30    0.039279
                ...   
2012-09-14   -0.002277
2012-08-14   -0.001861
2012-07-13   -0.022335
2012-06-13   -0.010616
2012-05-11    0.014634
2012-04-11    0.006285
2012-03-30   -0.026679
2012-02-29    0.025366
2012-01-20    0.013894
2011-12-20   -0.033575
2011-11-18    0.008411
2011-10-18   -0.015838
2011-09-16 

In [37]:
#market (HuShen300)
s_hs_market = get_price(['000300.XSHG'],start_date=data_starttime, end_date=data_endtime,frequency='1d',fields='close')
# df_Rm_data=pd.read_csv("df_Rm_data.csv",index_col=0,header=None)
type(df_hs_market)#pandas.core.series.Series
s_hs_market

2010-04-02    3407.3460
2010-04-06    3405.1450
2010-04-07    3386.9490
2010-04-08    3346.7440
2010-04-09    3379.1700
2010-04-12    3351.4780
2010-04-13    3391.7160
2010-04-14    3403.7070
2010-04-15    3394.5740
2010-04-16    3356.3320
2010-04-19    3176.4230
2010-04-20    3173.3740
2010-04-21    3236.6790
2010-04-22    3201.5410
2010-04-23    3190.0030
2010-04-26    3171.9970
2010-04-27    3108.4090
2010-04-28    3097.3530
2010-04-29    3060.0640
2010-04-30    3067.3650
2010-05-04    3019.4490
2010-05-05    3036.3940
2010-05-06    2896.8560
2010-05-07    2836.7890
2010-05-10    2858.2330
2010-05-11    2800.8170
2010-05-12    2818.1600
2010-05-13    2886.9110
2010-05-14    2868.0230
2010-05-17    2714.7150
                ...    
2018-02-12    3890.1046
2018-02-13    3935.6324
2018-02-14    3966.9614
2018-02-22    4052.7332
2018-02-23    4071.0889
2018-02-26    4118.4186
2018-02-27    4058.9806
2018-02-28    4023.6415
2018-03-01    4049.0863
2018-03-02    4016.4610
2018-03-05    40

In [33]:
#保存smb hml 因子
# smb.to_csv("smb.csv")
# hml.to_csv("hml.csv")
# s_hs_market.to_csv("hs_market.csv")

In [38]:
#处理无风险利率
df_rmb_save_ratio=pd.read_csv("rmb_ratio_pure.csv",index_col=0,header=None)
s_save_ratio=df_rmb_save_ratio[1]
s_save_ratio.index=pd.to_datetime(s_save_ratio.index)
# type(df_save_ratio.index[0])#pandas.tslib.Timestamp
s_save_ratio

0
1989-02-01    7.56
1990-04-15    6.30
1990-08-21    4.32
1991-04-21    3.24
1993-05-15    4.86
1993-07-11    6.66
1996-05-01    4.86
1996-08-23    3.33
1997-10-23    2.88
1998-03-25    2.88
1998-07-01    2.79
1998-12-07    2.79
1999-06-10    1.98
2002-02-21    1.71
2004-10-29    1.71
2006-08-19    1.80
2007-03-18    1.98
2007-05-19    2.07
2007-07-21    2.34
2007-08-22    2.61
2007-09-15    2.88
2007-12-21    3.33
2008-10-09    3.15
2008-10-30    2.88
2008-11-27    1.98
2008-12-23    1.71
2010-10-20    1.91
2010-12-26    2.25
2011-02-09    2.60
2011-04-06    2.85
2011-07-07    3.10
2012-06-08    2.85
2012-07-06    2.60
2014-11-22    2.35
2015-03-01    2.10
2015-05-11    1.85
2015-06-28    1.60
2015-08-26    1.35
2015-10-24    1.10
Name: 1, dtype: float64

In [44]:
#market return(HuShen300) and risk free return(0.375724091% 1M, for I can't get this rate from Ricequant nor can I upload my own data)
Rm = pd.Series(index = index_date)
Rf = pd.Series(index = index_date)
for i in range(data_count):
    if i==data_count-1:
        Rm[index_date[i]] = 0
        Rf[index_date[i]] = 0
        continue
    if(s_hs_market.loc[index_date[i+1]]!=0):
        Rf[index_date[i]]=s_save_ratio.asof(index_date[i])
        Rm[index_date[i]] = s_hs_market.loc[index_date[i]]/s_hs_market.loc[index_date[i+1]] - 1

Rf  

In [46]:
#save rm and rf data
Rm.to_csv("rm.csv")
Rf.to_csv("rf.csv")

In [60]:
#计算回归模型
# return_data['600000.XSHG'] - Rf
# 2010-11-01   -0.00375724 因为return_date为0，所有应该踢掉
# 2010-09-30    -0.0846042
y = np.array(return_data['600000.XSHG'] - Rf)
y=y[:data_count-1]
print(y)
YT = y.T
YT
y

[-1.1650080256821831 -1.1708426547352722 -1.0390822784810128
 -1.1343773873185639 -1.0821150855365476 -1.1023273855702096
 -1.0629927594529365 -1.1758364312267657 -1.0100486223662886
 -1.0179709763689773 -1.1059618753758858 -1.1672818178122841
 -1.1264432589872257 -1.0853601342723833 -1.1156128950655348
 -1.0963821583281763 -1.0914945710879427 -1.0963380259068607
 -1.0975607792625988 -1.0539632682963798 -1.1153477496600446
 -1.0853272757866399 -1.1128070962276375 -1.1121023654996431
 -1.1146337125162711 -0.98182434570356647 -1.1968803324597286
 -1.1203714367981572 -0.96210944965734413 -1.114432305401575
 -1.1821670996396549 -1.4156174144546236 -1.6384821705864085
 -1.9481259230338319 -1.8098389674138555 -1.9731695185037803
 -2.015688643424232 -2.471430669939441 -2.1107230104293904
 -2.1307308970099665 -2.4785971223021583 -2.5875130072840795
 -2.6041450777202071 -2.521693583587953 -2.5994235002271058
 -2.6174059324361441 -2.6336728261951365 -2.5619642887890217
 -2.5079781133068839 -2.63

array([-1.1650080256821831, -1.1708426547352722, -1.0390822784810128,
       -1.1343773873185639, -1.0821150855365476, -1.1023273855702096,
       -1.0629927594529365, -1.1758364312267657, -1.0100486223662886,
       -1.0179709763689773, -1.1059618753758858, -1.1672818178122841,
       -1.1264432589872257, -1.0853601342723833, -1.1156128950655348,
       -1.0963821583281763, -1.0914945710879427, -1.0963380259068607,
       -1.0975607792625988, -1.0539632682963798, -1.1153477496600446,
       -1.0853272757866399, -1.1128070962276375, -1.1121023654996431,
       -1.1146337125162711, -0.98182434570356647, -1.1968803324597286,
       -1.1203714367981572, -0.96210944965734413, -1.114432305401575,
       -1.1821670996396549, -1.4156174144546236, -1.6384821705864085,
       -1.9481259230338319, -1.8098389674138555, -1.9731695185037803,
       -2.015688643424232, -2.471430669939441, -2.1107230104293904,
       -2.1307308970099665, -2.4785971223021583, -2.5875130072840795,
       -2.60414507772

In [61]:
from statsmodels import regression
import statsmodels.api as sm

In [62]:
#x = pd.DataFrame(index=index_date,columns=['Rm-Rf','SMB','HML'])
Rm_Rf = np.array(Rm-Rf)
Rm_Rf = Rm_Rf[:data_count-1]
SMB = np.array(smb)
SMB = SMB[:data_count-1]
HML = np.array(hml)
HML = HML[:data_count-1]
X = np.column_stack((Rm_Rf,SMB,HML))

In [63]:
X

array([[ -1.13110212e+00,   1.53325202e-02,  -3.50672096e-02],
       [ -1.18163288e+00,   5.21345365e-02,  -3.35332316e-02],
       [ -1.01916435e+00,  -7.89530363e-02,   4.56214665e-02],
       [ -1.11232419e+00,  -7.27313241e-05,  -1.41775680e-02],
       [ -1.06342671e+00,  -1.71855358e-02,  -9.72382665e-03],
       [ -1.06829769e+00,  -2.27482976e-02,  -9.49274058e-03],
       [ -1.07723198e+00,  -2.28729924e-02,  -2.45364203e-03],
       [ -1.09364362e+00,  -2.31955154e-02,  -5.13550171e-03],
       [ -1.06080146e+00,  -4.34781200e-02,   2.92930158e-02],
       [ -1.04591247e+00,  -3.37070063e-02,   1.32565771e-02],
       [ -1.11219629e+00,   5.09679561e-03,  -1.24095475e-02],
       [ -1.09997940e+00,  -6.28504901e-03,  -4.10269122e-03],
       [ -1.09287926e+00,  -2.17420692e-02,  -1.98004330e-02],
       [ -1.07141449e+00,  -2.36553989e-02,   1.19541763e-02],
       [ -1.10587986e+00,   1.69561908e-03,   5.37473332e-03],
       [ -1.12436488e+00,   1.22312456e-02,  -8.7694861

In [64]:
X = sm.add_constant(X)
mod = regression.linear_model.OLS(YT, X).fit()
a = mod.params
print(a)

[0.015205901935623498 0.999432830688601 0.95303786247308175
 1.2577808724247903]


In [65]:
print(mod.tvalues)

[0.79396251632366577 120.73950859466034 3.3465672675340339
 2.5100704974875372]


In [29]:
# print(mod.t_test(np.array([1,0,0,0])))
# mod.summary("y","x","title")
# mod.describes()
# mod?

In [1]:
print(2)

2
